In [38]:
import pandas as pd
import pickle
from sklearn.ensemble import AdaBoostRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error

In [56]:
# Use only last 3 datasets for learning models

#df1 = pd.read_csv('datasets/db_waw_flats.csv', parse_dates = ['today', 'publication_date'])
#df2 = pd.read_csv('datasets/db_waw_flats_17_06.csv', parse_dates = ['today', 'publication_date'])
#df3 = pd.read_csv('datasets/db_waw_flats_27_6.csv', parse_dates = ['today', 'publication_date'])
#df4 = pd.read_csv('datasets/db_waw_flats_10_7.csv', parse_dates = ['today', 'publication_date'])
#df5 = pd.read_csv('datasets/db_waw_flats_27_7.csv', parse_dates = ['today', 'publication_date'])
#df6 = pd.read_csv('datasets/db_waw_flats_3_8.csv', parse_dates = ['today', 'publication_date'])
df7 = pd.read_csv('datasets/db_waw_flats_16_8.csv', parse_dates = ['today', 'publication_date'])
df8 = pd.read_csv('datasets/db_waw_flats_4_9.csv', parse_dates = ['today', 'publication_date'])
df9 = pd.read_csv('datasets/db_waw_flats_10_9.csv', parse_dates = ['today', 'publication_date'])

In [72]:
common_df = pd.concat([df7,df8,df9], ignore_index=True)
common_df

,price,area,rooms,renovation,floor,market,elevator,street,district,balcony,terrace,garden,parking,central_heating,seller,blok,cena_m,today,publication_date
0,599000.0,52.70,3.0,1,2,2,1,ul. heleny junkiewicz,targówek,0,0,0,0,1,1,1,11366.223909,2023-08-03,2023-08-03 14:36:55
1,1149900.0,80.00,3.0,3,2,2,1,al. aleja wilanowska,mokotów,1,0,0,1,1,1,0,14373.750000,2023-08-03,2023-07-01 07:51:29
2,620000.0,56.30,2.0,2,2,2,1,ul. motorowa,praga-południe,0,0,0,1,1,1,1,11012.433393,2023-08-03,2023-07-10 14:02:50
3,670000.0,61.70,2.0,3,1,2,1,ul. odkryta,białołęka,1,0,0,1,0,1,1,10858.995138,2023-08-03,2023-08-03 14:33:10
4,1546095.0,87.35,4.0,3,2,2,0,ul. zwierzyniecka,mokotów,0,0,0,0,0,1,0,17700.000000,2023-08-03,2023-07-18 18:55:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23128,1500000.0,105.40,4.0,3,2,2,1,szczęśliwice,ochota,1,0,0,1,1,1,0,14231.499051,2023-09-04,2016-07-05 18:02:25
23129,1479400.0,113.80,4.0,3,1,2,1,natolin,ursynów,1,0,0,1,1,1,1,13000.000000,2023-09-04,2015-10-27 20:02:18
23130,NaN,319.00,2.0,3,2,2,0,unknown,wesoła,0,0,0,1,0,1,0,NaN,2023-09-04,2015-02-06 23:10:16
23131,NaN,262.00,6.0,3,1,2,0,unknown,żoliborz,0,0,0,1,0,1,0,NaN,2023-09-04,2015-02-06 22:28:39


In [73]:
common_df.drop(columns=['today']).drop_duplicates(keep='last', inplace=True)

In [74]:
cols_x = ['district', 'area', 'rooms', 'renovation', 'floor', 'balcony', 'terrace',
          'garden', 'parking', 'central_heating', 'market', 'seller', 'blok', 'elevator']

cat_cols = ['district', 'renovation', 'floor', 'balcony', 'terrace', 'garden', 'parking',
            'central_heating', 'market', 'seller', 'blok', 'elevator']
col_y = ['price']

model_df = common_df[cols_x + col_y].dropna()

In [75]:
#le = preprocessing.LabelEncoder()
#le.fit(model_df['district'])
le = pickle.load(open('district_encode', 'rb'))
model_df['district'] = le.transform(model_df['district'])

In [76]:
X_train, X_test, y_train, y_test = train_test_split(model_df[cols_x], model_df['price'], test_size=0.2, shuffle=False)

In [77]:
from sklearn.tree import DecisionTreeRegressor
est = DecisionTreeRegressor(random_state=0, max_depth=12)
ada_model = AdaBoostRegressor(estimator=est, random_state=0, n_estimators=50, learning_rate=0.05, loss='square')
ada_model.fit(X_train, y_train)
ada_preds = ada_model.predict(X_test)

In [78]:
print('percentage error', mean_absolute_percentage_error(y_test, ada_preds))
print('absolute error', mean_absolute_error(y_test, ada_preds))
print('root squared error', mean_squared_error(y_test, ada_preds, squared=False))

percentage error 0.09926396986783727
absolute error 102928.37499949908
root squared error 233535.62713213387


In [79]:
# Catboost is fitting automatically during parsing new data

#from catboost import Pool, CatBoostRegressor
#cat_df = pd.concat([df6,df7,df8], ignore_index=True)
#cat_df.drop(columns=['today']).drop_duplicates(keep='last', inplace=True)
#cat_df = cat_df[cols_x + col_y].dropna()
#cat_df['district'] = le.transform(cat_df['district'])

#X_train, X_test, y_train, y_test = train_test_split(model_df[cols_x], model_df['price'], test_size=0.25, random_state=10)
#train_pool = Pool(X_train, y_train, cat_features=cat_cols)
#test_pool = Pool(X_test, cat_features=cat_cols)
#cb_model = CatBoostRegressor(iterations=3000, learning_rate=0.04, l2_leaf_reg=1, depth=8)
#cb_model.fit(train_pool)
#cb_preds = cb_model.predict(test_pool)

0:	learn: 1232831.8945596	total: 7.98ms	remaining: 23.9s
1:	learn: 1201337.6476142	total: 14.8ms	remaining: 22.1s
2:	learn: 1171881.2891714	total: 21.1ms	remaining: 21.1s
3:	learn: 1142957.4957660	total: 27.1ms	remaining: 20.3s
4:	learn: 1116304.6247689	total: 32.9ms	remaining: 19.7s
5:	learn: 1091448.3158691	total: 38.2ms	remaining: 19.1s
6:	learn: 1065928.5394505	total: 44.3ms	remaining: 18.9s
7:	learn: 1042327.5198944	total: 50.4ms	remaining: 18.9s
8:	learn: 1019312.1057534	total: 56.3ms	remaining: 18.7s
9:	learn: 998263.2397821	total: 62ms	remaining: 18.5s
10:	learn: 977247.0022766	total: 68ms	remaining: 18.5s
11:	learn: 958760.8465573	total: 74.4ms	remaining: 18.5s
12:	learn: 939137.0760371	total: 80.2ms	remaining: 18.4s
13:	learn: 921713.8805561	total: 86.6ms	remaining: 18.5s
14:	learn: 904117.9866336	total: 92.2ms	remaining: 18.4s
15:	learn: 886843.6147898	total: 98.6ms	remaining: 18.4s
16:	learn: 870785.1172996	total: 104ms	remaining: 18.3s
17:	learn: 854981.3822802	total: 110m

170:	learn: 416824.7552575	total: 1.02s	remaining: 16.8s
171:	learn: 416243.8421056	total: 1.03s	remaining: 16.9s
172:	learn: 415166.3265564	total: 1.03s	remaining: 16.9s
173:	learn: 414963.6884198	total: 1.04s	remaining: 16.9s
174:	learn: 414874.8589214	total: 1.04s	remaining: 16.9s
175:	learn: 414545.2875000	total: 1.05s	remaining: 16.8s
176:	learn: 413731.4364062	total: 1.06s	remaining: 16.8s
177:	learn: 412859.8725959	total: 1.06s	remaining: 16.8s
178:	learn: 412493.2590691	total: 1.07s	remaining: 16.8s
179:	learn: 411206.2975854	total: 1.07s	remaining: 16.8s
180:	learn: 410730.7351597	total: 1.08s	remaining: 16.8s
181:	learn: 410042.0921721	total: 1.09s	remaining: 16.8s
182:	learn: 410034.8010595	total: 1.09s	remaining: 16.8s
183:	learn: 409603.9167966	total: 1.09s	remaining: 16.8s
184:	learn: 409544.7078922	total: 1.1s	remaining: 16.7s
185:	learn: 409306.8663488	total: 1.1s	remaining: 16.7s
186:	learn: 409003.6833145	total: 1.11s	remaining: 16.7s
187:	learn: 408119.4560715	total:

337:	learn: 342118.5142861	total: 2.02s	remaining: 15.9s
338:	learn: 341686.9541910	total: 2.03s	remaining: 15.9s
339:	learn: 341453.4404513	total: 2.03s	remaining: 15.9s
340:	learn: 341187.7183788	total: 2.04s	remaining: 15.9s
341:	learn: 340938.1218513	total: 2.04s	remaining: 15.9s
342:	learn: 340719.4950964	total: 2.05s	remaining: 15.9s
343:	learn: 340485.0422168	total: 2.06s	remaining: 15.9s
344:	learn: 340248.6892472	total: 2.06s	remaining: 15.9s
345:	learn: 339887.0612115	total: 2.07s	remaining: 15.9s
346:	learn: 339539.4628082	total: 2.07s	remaining: 15.9s
347:	learn: 339486.8075472	total: 2.08s	remaining: 15.8s
348:	learn: 339306.2848237	total: 2.08s	remaining: 15.8s
349:	learn: 339050.0244657	total: 2.09s	remaining: 15.8s
350:	learn: 338996.6616147	total: 2.1s	remaining: 15.8s
351:	learn: 338655.6590170	total: 2.1s	remaining: 15.8s
352:	learn: 337775.9240159	total: 2.11s	remaining: 15.8s
353:	learn: 337062.0646121	total: 2.12s	remaining: 15.8s
354:	learn: 336197.0604200	total:

498:	learn: 298245.7436126	total: 3.02s	remaining: 15.1s
499:	learn: 298152.8021284	total: 3.02s	remaining: 15.1s
500:	learn: 297793.0054196	total: 3.03s	remaining: 15.1s
501:	learn: 297470.4159790	total: 3.04s	remaining: 15.1s
502:	learn: 297184.2557347	total: 3.04s	remaining: 15.1s
503:	learn: 296877.2691555	total: 3.05s	remaining: 15.1s
504:	learn: 296703.0624928	total: 3.06s	remaining: 15.1s
505:	learn: 296614.2392339	total: 3.06s	remaining: 15.1s
506:	learn: 296393.2001830	total: 3.07s	remaining: 15.1s
507:	learn: 296030.4922115	total: 3.07s	remaining: 15.1s
508:	learn: 295922.1208736	total: 3.08s	remaining: 15.1s
509:	learn: 295885.0587033	total: 3.09s	remaining: 15.1s
510:	learn: 295500.9460321	total: 3.09s	remaining: 15.1s
511:	learn: 295350.5018358	total: 3.1s	remaining: 15.1s
512:	learn: 295166.5844180	total: 3.1s	remaining: 15.1s
513:	learn: 295091.8207411	total: 3.11s	remaining: 15s
514:	learn: 294759.1603759	total: 3.12s	remaining: 15s
515:	learn: 294616.8246845	total: 3.1

661:	learn: 273424.5961548	total: 4.01s	remaining: 14.2s
662:	learn: 273301.9094611	total: 4.02s	remaining: 14.2s
663:	learn: 273196.7937479	total: 4.03s	remaining: 14.2s
664:	learn: 273111.1579709	total: 4.03s	remaining: 14.2s
665:	learn: 273008.6315775	total: 4.04s	remaining: 14.2s
666:	learn: 272843.8619005	total: 4.04s	remaining: 14.1s
667:	learn: 272773.0328652	total: 4.05s	remaining: 14.1s
668:	learn: 272352.2006259	total: 4.06s	remaining: 14.1s
669:	learn: 272209.7682056	total: 4.06s	remaining: 14.1s
670:	learn: 272133.1074807	total: 4.07s	remaining: 14.1s
671:	learn: 272073.2604724	total: 4.07s	remaining: 14.1s
672:	learn: 271938.3083465	total: 4.08s	remaining: 14.1s
673:	learn: 271846.0499313	total: 4.08s	remaining: 14.1s
674:	learn: 271751.0802892	total: 4.09s	remaining: 14.1s
675:	learn: 271668.8595261	total: 4.1s	remaining: 14.1s
676:	learn: 271608.9107830	total: 4.1s	remaining: 14.1s
677:	learn: 271045.6210208	total: 4.11s	remaining: 14.1s
678:	learn: 270979.9729052	total:

...

2288:	learn: 190800.6121790	total: 14s	remaining: 4.36s
2289:	learn: 190792.1022010	total: 14s	remaining: 4.36s
2290:	learn: 190788.9233277	total: 14.1s	remaining: 4.35s
2291:	learn: 190782.3442484	total: 14.1s	remaining: 4.34s
2292:	learn: 190774.8686159	total: 14.1s	remaining: 4.34s
2293:	learn: 190303.7218279	total: 14.1s	remaining: 4.33s
2294:	learn: 190257.1725579	total: 14.1s	remaining: 4.33s
2295:	learn: 190251.1369821	total: 14.1s	remaining: 4.32s
2296:	learn: 190236.2345340	total: 14.1s	remaining: 4.31s
2297:	learn: 190164.8514720	total: 14.1s	remaining: 4.31s
2298:	learn: 190056.0271595	total: 14.1s	remaining: 4.3s
2299:	learn: 190018.2321907	total: 14.1s	remaining: 4.29s
2300:	learn: 190008.2597350	total: 14.1s	remaining: 4.29s
2301:	learn: 189987.5947840	total: 14.1s	remaining: 4.28s
2302:	learn: 189969.5843425	total: 14.1s	remaining: 4.28s
2303:	learn: 189953.8072824	total: 14.1s	remaining: 4.27s
2304:	learn: 189940.4850772	total: 14.1s	remaining: 4.26s
2305:	learn: 189910

2452:	learn: 185196.9988396	total: 15.1s	remaining: 3.36s
2453:	learn: 185181.2424999	total: 15.1s	remaining: 3.35s
2454:	learn: 185172.3426497	total: 15.1s	remaining: 3.35s
2455:	learn: 185160.3888418	total: 15.1s	remaining: 3.34s
2456:	learn: 185143.7025854	total: 15.1s	remaining: 3.33s
2457:	learn: 185102.0481378	total: 15.1s	remaining: 3.33s
2458:	learn: 185044.7533601	total: 15.1s	remaining: 3.32s
2459:	learn: 185040.2563237	total: 15.1s	remaining: 3.32s
2460:	learn: 185028.0141414	total: 15.1s	remaining: 3.31s
2461:	learn: 184774.9145891	total: 15.1s	remaining: 3.3s
2462:	learn: 184756.2002404	total: 15.1s	remaining: 3.3s
2463:	learn: 184735.7310195	total: 15.1s	remaining: 3.29s
2464:	learn: 184726.6712676	total: 15.1s	remaining: 3.29s
2465:	learn: 184712.6473259	total: 15.1s	remaining: 3.28s
2466:	learn: 184703.4380199	total: 15.1s	remaining: 3.27s
2467:	learn: 184678.2036361	total: 15.2s	remaining: 3.27s
2468:	learn: 184667.2332401	total: 15.2s	remaining: 3.26s
2469:	learn: 184

2613:	learn: 180534.6442940	total: 16.1s	remaining: 2.37s
2614:	learn: 180512.0594203	total: 16.1s	remaining: 2.37s
2615:	learn: 180494.9185285	total: 16.1s	remaining: 2.36s
2616:	learn: 180474.5760536	total: 16.1s	remaining: 2.35s
2617:	learn: 180439.6080123	total: 16.1s	remaining: 2.35s
2618:	learn: 180391.1056533	total: 16.1s	remaining: 2.34s
2619:	learn: 180373.2555735	total: 16.1s	remaining: 2.33s
2620:	learn: 180308.2047975	total: 16.1s	remaining: 2.33s
2621:	learn: 180270.1027894	total: 16.1s	remaining: 2.32s
2622:	learn: 180257.2950152	total: 16.1s	remaining: 2.32s
2623:	learn: 180023.7960578	total: 16.1s	remaining: 2.31s
2624:	learn: 179994.9682341	total: 16.1s	remaining: 2.3s
2625:	learn: 179985.5884467	total: 16.1s	remaining: 2.3s
2626:	learn: 179970.5748765	total: 16.1s	remaining: 2.29s
2627:	learn: 179957.4423087	total: 16.2s	remaining: 2.29s
2628:	learn: 179940.6486350	total: 16.2s	remaining: 2.28s
2629:	learn: 179915.7666800	total: 16.2s	remaining: 2.27s
2630:	learn: 179

2770:	learn: 176664.0199593	total: 17.1s	remaining: 1.41s
2771:	learn: 176623.3046149	total: 17.1s	remaining: 1.4s
2772:	learn: 176612.4320461	total: 17.1s	remaining: 1.4s
2773:	learn: 176595.4951821	total: 17.1s	remaining: 1.39s
2774:	learn: 176561.6960969	total: 17.1s	remaining: 1.39s
2775:	learn: 176546.1689184	total: 17.1s	remaining: 1.38s
2776:	learn: 176533.4867585	total: 17.1s	remaining: 1.37s
2777:	learn: 176530.9431195	total: 17.1s	remaining: 1.37s
2778:	learn: 176492.9480343	total: 17.1s	remaining: 1.36s
2779:	learn: 176463.2341134	total: 17.1s	remaining: 1.35s
2780:	learn: 176450.6720166	total: 17.1s	remaining: 1.35s
2781:	learn: 176374.3586617	total: 17.1s	remaining: 1.34s
2782:	learn: 176357.0428392	total: 17.1s	remaining: 1.34s
2783:	learn: 176344.8338690	total: 17.1s	remaining: 1.33s
2784:	learn: 176336.2845699	total: 17.1s	remaining: 1.32s
2785:	learn: 176315.6791604	total: 17.2s	remaining: 1.32s
2786:	learn: 176278.2589915	total: 17.2s	remaining: 1.31s
2787:	learn: 176

2929:	learn: 172341.1919601	total: 18.1s	remaining: 432ms
2930:	learn: 172323.2099859	total: 18.1s	remaining: 426ms
2931:	learn: 172311.5925603	total: 18.1s	remaining: 419ms
2932:	learn: 172279.5690208	total: 18.1s	remaining: 413ms
2933:	learn: 172218.1961871	total: 18.1s	remaining: 407ms
2934:	learn: 172194.2570443	total: 18.1s	remaining: 401ms
2935:	learn: 172176.4646463	total: 18.1s	remaining: 395ms
2936:	learn: 172150.2542803	total: 18.1s	remaining: 389ms
2937:	learn: 172118.7660558	total: 18.1s	remaining: 382ms
2938:	learn: 172115.3835127	total: 18.1s	remaining: 376ms
2939:	learn: 172111.1977161	total: 18.1s	remaining: 370ms
2940:	learn: 172101.5979511	total: 18.1s	remaining: 364ms
2941:	learn: 172085.7459532	total: 18.1s	remaining: 358ms
2942:	learn: 172035.4291327	total: 18.2s	remaining: 352ms
2943:	learn: 172027.2131353	total: 18.2s	remaining: 345ms
2944:	learn: 172009.5321921	total: 18.2s	remaining: 339ms
2945:	learn: 171998.3180506	total: 18.2s	remaining: 333ms
2946:	learn: 1

In [80]:
print('percentage error', mean_absolute_percentage_error(y_test, cb_preds))
print('absolute error', mean_absolute_error(y_test, cb_preds))
print('root squared error', mean_squared_error(y_test, cb_preds, squared=False))

percentage error 0.10528519210166891
absolute error 125508.46901349057
root squared error 310599.1791421675


In [99]:
type(y_test)

pandas.core.series.Series

In [85]:
lr_df = pd.DataFrame(data=ada_preds.reshape(-1,1), columns=['ada_preds'])
lr_df['cb_preds'] = cb_preds
lr_df['y'] = y_test.values
train = lr_df[:2500]
test = lr_df[2500:]

In [87]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train.drop(columns=['y']), train['y'])
lr_preds = lr.predict(test.drop(columns=['y']))

In [88]:
print('percentage error', mean_absolute_percentage_error(test['y'], lr_preds))
print('absolute error', mean_absolute_error(test['y'], lr_preds))
print('root squared error', mean_squared_error(test['y'], lr_preds, squared=False))

percentage error 0.09640491419926653
absolute error 115799.59107391411
root squared error 272925.3933532612


In [89]:
lr.fit(lr_df.drop(columns=['y']), lr_df['y'])

LinearRegression()

In [90]:
import pickle
filename = 'adaboost_regressor.pkl'
pickle.dump(ada_model, open(filename, 'wb'))
#pickle.dump(le, open('district_encode', 'wb'))
filename = 'linear_regressor.pkl'
pickle.dump(lr, open(filename, 'wb'))
cb_model.save_model('cb_model.cbm')

In [86]:
load_model = pickle.load(open('district_encode', 'rb'))
load_model.transform(['praga-południe'])

array([5])

In [3]:
import sqlite3
con = sqlite3.connect("instance/test_flat.db")
#con.execute("DELETE FROM price_metric")
#con.commit()
res = con.execute("SELECT price_m2, date FROM test_metric ORDER BY id DESC").fetchall()
res[0]

(16273.838542763531, '2023-08-16 00:00:00.000000')

In [115]:
from pathlib import Path
THIS_FOLDER = Path(__file__).parent.resolve()
my_file = THIS_FOLDER / "myfile.txt"

NameError: name '__file__' is not defined

In [116]:
import os
print(__file__)

NameError: name '__file__' is not defined